In [ ]:
clear

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, regularizers, optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# 시간정보를 활용하여 폴더 생성
import datetime 

In [ ]:
#3채널 525*700 픽셀의 500개 이미지
IMG_CHANNELS = 3
IMG_ROWS = 525
IMG_COLS = 700
IMG_SIZE = (IMG_ROWS, IMG_COLS)

In [ ]:
VALIDATION_SPLIT = 0.10

#img_gen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split= VALIDATION_SPLIT)
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=VALIDATION_SPLIT) 

train_set = img_gen.flow_from_directory(directory="/content/drive/MyDrive/dataset/Generated_Spectograms",
                                        target_size = IMG_SIZE,                                                     
                                        class_mode='categorical',
                                        subset='training', 
                                        batch_size = 500)

test_set = img_gen.flow_from_directory(directory="/content/drive/MyDrive/dataset/Generated_Spectograms/",
                                        target_size = IMG_SIZE,
                                        class_mode='categorical',
                                        subset='validation',
                                        shuffle = False,
                                        batch_size = 500)

# 이미지와 이미지에 해당하는 label
Train_X, Train_Y = next(train_set)
Test_X, Test_Y = next(test_set)

In [ ]:
# 그래프만 추출
X_train = Train_X[:, 39:467, 81:567, :]
X_test = Test_X[:, 39:467, 81:567, :]

In [ ]:
# image augmentation: random_crop
def random_crop(img_set, random_crop_width):
    height, width = img_set.shape[1], img_set.shape[2]
    dx = random_crop_width
    x = np.random.randint(0, width - dx + 1)
    return img_set[ :, :, x:(x+dx), :]

In [ ]:
crop_width = 81

argumented_X_train = random_crop(X_train, crop_width)
argumented_Y_train = Train_Y

for i in range(9):
  new_set = random_crop(X_train, crop_width)
  argumented_X_train = np.concatenate((argumented_X_train, new_set),axis=0)
  argumented_Y_train = np.concatenate((argumented_Y_train,Train_Y),axis=0)

argumented_X_test = random_crop(X_test, crop_width)
argumented_Y_test = Test_Y

for i in range(9):
  new_set = random_crop(X_test, crop_width)
  argumented_X_test = np.concatenate((argumented_X_test, new_set),axis=0)
  argumented_Y_test = np.concatenate((argumented_Y_test, Test_Y),axis=0)


In [ ]:
argumented_X_train = tf.image.resize(argumented_X_train, (428, 81))
argumented_X_test = tf.image.resize(argumented_X_test, (428, 81))

# 형식 변환
#X_train = np.array(X_train.astype('float32'))
#X_test = np.array(X_test.astype('float32'))
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [ ]:
# 상수
BATCH_SIZE =  100
EPOCHS = 50
NUM_CLASSES = 5
VERBOSE = 1
OPTIM = tf.keras.optimizers.RMSprop()

INPUT_SHAPE = (428, 81, 3)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, Reshape, LSTM

model = Sequential()
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape= INPUT_SHAPE))
model.add(Conv1D(32, kernel_size=3, activation='relu'))

model.add(Reshape((428*77, 32)))
model.add(LSTM(16, return_sequences=True))
model.add(Flatten())
model.add(Dense(5, activation='softmax'))
model.summary()
OPTIM = tf.keras.optimizers.RMSprop()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
# 텐서보드 사용
callbacks = [
    # 텐서보드 로그를 './logs' 디렉터리에 작성
    tf.keras.callbacks.TensorBoard(log_dir = './logs')
]

# 학습데이터의 log를 저장할 폴더 생성 (지정)
log_dir = "logs/my_board/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# 텐서보드 콜백 정의 하기
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [ ]:
history = model.fit(argumented_X_train, argumented_Y_train,
                    batch_size = BATCH_SIZE, epochs = EPOCHS,
                    verbose = VERBOSE, validation_split = VALIDATION_SPLIT,
                    callbacks = [tensorboard_callback])

score = model.evaluate(argumented_X_test, argumented_Y_test, batch_size = BATCH_SIZE)
print("\nTest score:", score[0])
print("Test accuracy:", score[1])